In [1]:
#ignore warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import yfinance, pandas and os
import yfinance as yf
import pandas as pd
import datetime
import os
import yesg

In [3]:
# Tickers, Forbes Just 100 top esg companies with all financial and esg details are available on yahoo finance
Symbols = ["INTC", "MSFT", "AAPL", "NVDA", "VZ", "CSCO", "EXC", "PEP", "MA", "ANTM", "ACN", "T", "IBM",
           "F", "OC", "INTU", "MRK", "AEP", "UNH", "CI", "AMAT", "CL", "EQIX", "UPS", "NEM", "LMT", "BLK",
           "EIX", "JNJ", "GS", "NOC", "PG", "ADM", "ETR", "QCOM", "GIS", "DE", "LOW", "AMGN", "ABT",
            "BDX", "CLX", "AES", "SPGI", "ES", "HSY", "AXP", "NLSN", "EBAY", "ECL", "NDAQ", "DLR", "BBY", 
           "KMB", "EMN"]

In [4]:
def create_esg_full(Symbols):
    esg_data = pd.DataFrame()
    
    for i in Symbols:
        try:
            temp =pd.DataFrame(yesg.get_esg_full(i))
            
            # append the individual stock info
            if len(temp) == 0:
                None
            else:
                esg_data = esg_data.append(temp, ignore_index = True) 
#                 print(f'{i}: esg appended')
        
        except Exception:
            None
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct     
    esg_data['Total-Score'] = pd.to_numeric(esg_data['Total-Score'])
    esg_data['E-Score'] = pd.to_numeric(esg_data['E-Score'])
    esg_data['S-Score'] = pd.to_numeric(esg_data['S-Score'])
    esg_data['G-Score'] = pd.to_numeric(esg_data['G-Score'])
    esg_data['Last Rated']= pd.to_datetime(esg_data['Last Rated'])
    esg_data.drop(['environment Percentile','social Percentile', 'governance Percentile'], axis = 1, inplace = True)
    esg_data.loc[esg_data['Controversial Business Areas'] == "", "Controversial Business Areas"] = 'None'
    esg_data.loc[esg_data['related Controversy'] == "-", "related Controversy"] = 'None'
    esg_data['ESG Performance'].fillna('Not Available', inplace = True)
    esg_data.insert(0, "Ticker", esg_data.pop('Ticker'))
    esg_data.rename(columns={'Last Rated': 'lastRated', 'ESG Performance': 'esgPerformance','peer Group': 'peerGroup',
                             "Highest Controversy" : "highestControversy", "peer Count": "peerCount",
                             "total Percentile":"totalPercentile", "related Controversy": "relatedControversy",
                             'min peer ESG': "minPeerESG", "avg peer ESG": "avgPeerESG", "max peer ESG":"maxPeerESG",
                             'min peer Environment': "minPeerEnvironment", "avg peer Environment": "avgPeerEnvironment",
                             "max peer Environment":"maxPeerEnvironment",'min peer Social': "minPeerSocial", 
                             "avg peer Social": "avgPeerSocial", "max peer Social":"maxPeerSocial",
                             'min peer Governance': "minPeerGovernance", "avg peer Governance": "avgPeerGovernance",
                             "max peer Governance":"maxPeerGovernance", "min Highest Controversy": "minHighestControversy",
                             "avg Highest Controversy": "avgHighestControversy", "max Highest Controversy": "maxHighestControversy",
                             "Controversial Business Areas" : "controversialBusinessAreas",'Total-Score': 'totalScore',
                             'E-Score': 'eScore', 'S-Score': 'sScore', 'G-Score': 'gScore'
                             }, inplace=True)
    
    
    return esg_data
    

In [5]:
esg_data = create_esg_full(Symbols)
print(f'total stocks in esg_full is : {len(esg_data)}')

total stocks in esg_full is : 55


In [6]:
# Created a function to extract stocks description from yahoo finance
def create_info_list(Symbols):
    # create empty list
    info = []
    # iterate over each symbol
    for i in Symbols:  
        try:
            # download stock info
            temp = []
            temp = yf.Ticker(i).info
                        
            # append the individual stock info
            if len(temp) == 0:
                None
            else:
                info.append(temp)
#                 print(f'{i}: info appended')
                
        except Exception:
            None
   
    return info


In [7]:
def create_info(Symbols):
    info_df = pd.DataFrame()
    info = create_info_list(Symbols)
#     print(len(info))
#     print(len(Symbols))
    if len(info) == 0:
        None
    else:
        info_df = info_df.append(info,sort=False, ignore_index=True)
        
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
    info_df = info_df[['symbol','longName', 'sector','industry', 'city', 'state','country','fullTimeEmployees',
                       'revenueGrowth', 'forwardEps', 'beta' , 'priceToSalesTrailing12Months' , 'forwardPE', 
                       'dividendRate', 'trailingPE', 'bookValue', 'returnOnAssets', 'returnOnEquity','averageVolume', 'volume','currentPrice',
                       'fiftyTwoWeekLow','fiftyTwoWeekHigh', 'regularMarketPrice', 'marketCap']]
    info_df.rename(columns = {'symbol':'Ticker'}, inplace = True)
    info_df.loc[info_df['country'] == 'Ireland', 'state'] = 'Leinster'
    info_df['industry'] = esg_data['peerGroup']
    return info_df

In [8]:
info_df = create_info(esg_data['Ticker'].to_list())
print(f'total stocks in stock_details is : {len(info_df)}')

total stocks in stock_details is : 55


In [9]:
info_df['industry'] = esg_data['peerGroup']

In [10]:
# function to get the time based data from yahoo finance and storing it in a Dataframe
def create_date_based_dataframes(Symbols,start, end):
    # create empty dataframe
    stocks_df = pd.DataFrame()
    # iterate over each symbol
    for i in Symbols:  

        # print the symbol which is being downloaded
        print( str(Symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  

        try:
            # download the stock price 
            stock = []
            stock = yf.download(i,start=start, end=end, progress=False)

            # append the individual stock prices 
            if len(stock) == 0:
                None
            else:
                stock['Ticker']=i
                stocks_df = stocks_df.append(stock,sort=False)
                print('data appended')
                
        except Exception:
            None
    # Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
    stocks_df.dropna(inplace =True)
    stocks_df = stocks_df.rename_axis('Date').reset_index()
    stocks_df.insert(0, "Ticker", stocks_df.pop('Ticker'))
    return stocks_df

In [11]:
# Time series data start and end date YYYY,MM,DD
start = datetime.datetime(2017,3,16)
end = datetime.datetime(2022,3,16)
stock_historical_df = create_date_based_dataframes(Symbols,start, end)

0 : INTC,data appended
1 : MSFT,data appended
2 : AAPL,data appended
3 : NVDA,data appended
4 : VZ,data appended
5 : CSCO,data appended
6 : EXC,data appended
7 : PEP,data appended
8 : MA,data appended
9 : ANTM,data appended
10 : ACN,data appended
11 : T,data appended
12 : IBM,data appended
13 : F,data appended
14 : OC,data appended
15 : INTU,data appended
16 : MRK,data appended
17 : AEP,data appended
18 : UNH,data appended
19 : CI,data appended
20 : AMAT,data appended
21 : CL,data appended
22 : EQIX,data appended
23 : UPS,data appended
24 : NEM,data appended
25 : LMT,data appended
26 : BLK,data appended
27 : EIX,data appended
28 : JNJ,data appended
29 : GS,data appended
30 : NOC,data appended
31 : PG,data appended
32 : ADM,data appended
33 : ETR,data appended
34 : QCOM,data appended
35 : GIS,data appended
36 : DE,data appended
37 : LOW,data appended
38 : AMGN,data appended
39 : ABT,data appended
40 : BDX,data appended
41 : CLX,data appended
42 : AES,data appended
43 : SPGI,data appende

In [12]:
# Creating business business_controversy_df from esg_data to know more about business values lowering ESG scores

# Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
business_controversy_df = esg_data['controversialBusinessAreas'].str.get_dummies(sep=', ')
business_controversy_df['Ticker'] = esg_data['Ticker']
business_controversy_df.insert(0, "Ticker", business_controversy_df.pop('Ticker'))
business_controversy_df.rename(columns={'Animal Testing': 'animalTesting', 'Catholic Values': 'catholicValues',
                                        'Controversial Weapons': 'controversialWeapons', 'Military Contracting': 'militaryContracting',
                                        'Small Arms': 'smallArms', 'Thermal Coal': 'thermalCoal'
                            }, inplace=True)


In [13]:
# Creating business controversy_type_df from esg_data to know more about controversies lowering ESG scores

# Data cleaning to ensure table names are proper, no null or duplicate data and datatypes are correct   
controversy_type_df = esg_data['relatedControversy'].str.get_dummies(sep=',')
controversy_type_df['Ticker'] = esg_data['Ticker']
controversy_type_df.insert(0, "Ticker", controversy_type_df.pop('Ticker'))
controversy_type_df.rename(columns={"Business Ethics Incidents": "businessEthicsIncidents", "Customer Incidents":"customerIncidents",
                                    "Employee Incidents":"employeeIncidents", "Environmental Supply Chain Incidents":"environmentalSupplyChainIncidents",
                                    "Governance Incidents" :"governanceIncidents","Operations Incidents":"operationsIncidents",
                                    "Product & Service Incidents" :"productServiceIncidents" , "Public Policy Incidents": "publicPolicyIncidents",
                                    "Social Supply Chain Incidents": "socialSupplyChainIncidents", "Society & Community Incidents" :"societyCommunityIncidents" 
                            }, inplace=True)


In [14]:
# Save full_esg
file_to_save = os.path.join("Resources", "esg_details.csv")
esg_data.to_csv(file_to_save, index = False)

# Save full_info
file_to_save = os.path.join("Resources", "stocks_details.csv")
info_df.to_csv(file_to_save, index = False)

# Save stocks historical data
file_to_save = os.path.join("Resources", "stocks_historical_data.csv")
stock_historical_df.to_csv(file_to_save, index = False)

# Save business controversy in esg data
file_to_save = os.path.join("Resources", "business_controversy_esg.csv")
business_controversy_df.to_csv(file_to_save, index = False)

# Save related controversy in esg data
file_to_save = os.path.join("Resources", "controversy_type_esg.csv")
controversy_type_df.to_csv(file_to_save, index = False)
